In [69]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for TensorFlow
import numpy as np
import tflearn
import tensorflow as tf
import random

# import our helper file
import chatbot

In [71]:
ints = chatbot.Intents()
for el in ints.intents:
    print(el)

{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there']}
{'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again soon.']}
{'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}
{'tag': 'med', 'patterns': ["I'm looking for cheap meds", 'want to find a deal', 'where are the cheapest meds', 'where can I buy meds for less money'], 'responses': ["What's the name of the medication?"], 'context': 'getRx'}
{'tag': 'handleRx', 'patterns': [], 'responses': [], 'context': 'handleRx'}
{'tag': 'coupon', 'patterns': ['what is the coupon', 'send me the coupon'], 'responses': ['Sorry, no coupon', 'No coupon available'], 'context': 'coupon'}


In [75]:
ints = chatbot.Intents()
print(ints.intents)

NameError: name 'intent' is not defined

With our intents JSON file loaded, we can now begin to organize our documents, words and classification classes.

In [44]:
words, documents, classes = intents.organize_intents()

AttributeError: 'Intents' object has no attribute 'organize_patterns'

In [15]:
def stem_words(words):
    # stem and lower each word and remove duplicates. set removes duplicates, then turn back to list
    return [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
    
def remove_duplicates(ls):
    return sorted(list(set(ls)))

# remove duplicates
classes = sorted(list(set(classes)))

# words is our word lexicon

17 documents
5 classes ['coupon', 'goodbye', 'greeting', 'med', 'thanks']
40 unique stemmed words ["'m", "'s", 'a', 'anyon', 'ar', 'buy', 'bye', 'can', 'cheap', 'cheapest', 'coupon', 'day', 'deal', 'find', 'for', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'lat', 'less', 'look', 'me', 'med', 'money', 'see', 'send', 'thank', 'that', 'the', 'ther', 'to', 'want', 'what', 'wher', 'you']


We create a list of documents (sentences), each sentence is a list of stemmed words and each document is associated with an intent (a class). The stem 'tak' will match 'take', 'taking', 'takers', etc. We could clearn the words list and remove useless entries but this will suffice for now. 

Unfortunately, this data structure won't work with TensorFlow, we need to transform it further: *from documents of words* into *tensors of numbers*.

Python List index() method: The method __index()__ returns the lowest inde in list that obj appears.

In [17]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag
    # list is not necessary because output_empty is already a list
    # index function: 
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

Notice that our data is shuffled. TensorFlow will take some of this and use it as test data to *gauge accuracy for a newly fitted model*.

If we look at a single x and y list element, we see 'bag of words' arrays, one for the intent pattern, the other for the intent class. We're ready to build our model.

In [21]:
# reset underlying graph data
tf.reset_default_graph()
tflearn.config.init_training_mode()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')


Training Step: 2999  | total loss: 0.03238 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.03238 - acc: 1.0000 -- iter: 16/17
Training Step: 3000  | total loss: 0.02946 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.02946 - acc: 1.0000 -- iter: 17/17
--
INFO:tensorflow:C:\Users\kraus_ju\Documents\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


To complete this section of work, we'll save ('pickle') our model and documents so the next notebook can use them.

In [19]:
# save all of our data structures
import pickle
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y},
           open("training_data", "wb"))